In [1]:
#pip install spotipy --upgrade              # Uncomment this and run it if you haven't installed spotipy before


In [2]:
# Dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import re, glob
import os, sys
from scipy import stats

import spotipy                          # Set up a config file with ckey and skey. These are available if you go
from spotipy.oauth2 import SpotifyClientCredentials      # to https://developer.spotify.com/, click on Dashboard 
                                        # from the horizontal black menu, login with your normal user info. Click 
# Import Keys                           # on "create an app" if you haven't yet, it doesn't matter what you call it.
from config import clientID, clientSEC           # Then click into
                                        # your project and you should see Client ID and Client Secret. Those are your 
                                        # ckey and skey. 

In [3]:
# Setting up Spotify API info

client_credentials_manager = SpotifyClientCredentials(client_id=clientID, client_secret=clientSEC)

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)



TIP: This next section assumes that you have already downloaded the csv files with the Top200 charts for the country you are working on:

1. Create a subfolder in the folder this notebook is located in called "input_files". Add the owid-covid-data.csv file there, you'll need that later. Then make another subfolder inside input_files called "spotify_top200_charts". Save the csv files you download there. 
2. Go to https://spotifycharts.com
2. Choose the country you want to work on. 
3. Download Weekly Top200 charts for 2019 and 2020, 1 chart per month. We agreed as a group to download the chart from last week of each month, to keep things consistent. Save them in the "spotify_top200_charts" folder you set up. 

In [4]:
# Create dataframe from weekly chart data


path = r"input_files/spotify_top200_charts/*.csv"                # The path requires "".csv" at end of file name. 
                                                                 # This is to prevent the program from blowing up
                                                                 # when it hits some kind of hidden file. 

country_tracks_df = pd.read_csv(glob.glob(path)[0], header=1)    # Sets up **main dataframe** with data from **FIRST** file
string = str(glob.glob(path)[0])                                 # in the folder.
year_month, = re.findall(r"ly-(\d\d\d\d)-(\d\d)-\d\d", string)   # This line extracts the year and month from the 
                                                                 # **file name**
country_tracks_df[['Year']] = year_month[0]
country_tracks_df[['Month']] = year_month[1]
country_tracks_df[['yyyy-mm']] = str(year_month[0]) + "-" + str(year_month[1]) 
Tot_Streams1 = country_tracks_df['Streams'].sum()                # Find out total streams in FIRST file in folder.
country_tracks_df[['Stream %']] = country_tracks_df['Streams'] / Tot_Streams1    # New column with % of streams

for file in glob.glob(path)[1:]:                                 # Now that you have the dataframe set up from the 
        temp_df = pd.read_csv(file, header=1)                    # first file in the folder, this iterates through 
        string = str(file)                                       # remaining files
        year_month, = re.findall(r"ly-(\d\d\d\d)-(\d\d)-\d\d", string)
        #print (year_month)
        Tot_Streams2 = temp_df['Streams'].sum()
        temp_df[['Year']] = year_month[0]
        temp_df[['Month']] = year_month[1]
        temp_df[['yyyy-mm']] = str(year_month[0]) + "-" + str(year_month[1])
        temp_df[['Stream %']] = temp_df['Streams'] / Tot_Streams2
        country_tracks_df = pd.concat([country_tracks_df,        # Adds temperary datafame to end of main dataframe
                                       temp_df])                 # as new rows.  ¿¿¿???? 
        

                                                 
country_tracks_df = country_tracks_df.sort_values(['Year','Month'])  # Sort the new dataframe by year and month




                                                                 # You should get 4,800 rows (24 months x 200 
                                                                 # tracks per month)
country_tracks_df

,Position,Track Name,Artist,Streams,URL,Year,Month,yyyy-mm,Stream %
0,1,MIDDLE CHILD,J. Cole,20663232,https://open.spotify.com/track/2JvzF1RMd7lE3Km...,2019,01,2019-01,0.033351
1,2,7 rings,Ariana Grande,20288575,https://open.spotify.com/track/14msK75pk3pA33p...,2019,01,2019-01,0.032746
2,3,Sunflower - Spider-Man: Into the Spider-Verse,Post Malone,13134644,https://open.spotify.com/track/3KkXRkHbMCARz0a...,2019,01,2019-01,0.021199
3,4,Wow.,Post Malone,12578954,https://open.spotify.com/track/6MWtB6iiXyIwun0...,2019,01,2019-01,0.020303
4,5,Going Bad (feat. Drake),Meek Mill,9707256,https://open.spotify.com/track/2IRZnDFmlqMuOrY...,2019,01,2019-01,0.015668
...,...,...,...,...,...,...,...,...,...
195,196,evermore (feat. Bon Iver),Taylor Swift,1581683,https://open.spotify.com/track/0AaV4MJrgmLV3Xh...,2020,12,2020-12,0.003181
196,197,Intentions (feat. Quavo),Justin Bieber,1581498,https://open.spotify.com/track/4umIPjkehX1r7uh...,2020,12,2020-12,0.003180
197,198,Beautiful Crazy,Luke Combs,1581158,https://open.spotify.com/track/2rxQMGVafnNaRaX...,2020,12,2020-12,0.003179
198,199,Party Girl,StaySolidRocky,1580003,https://open.spotify.com/track/3IJYUaN3tx04S4T...,2020,12,2020-12,0.003177


In [ ]:
# Get Track IDs
track_names = country_tracks_df['Track Name'].to_list()           # Set up list of tracks to iterate through         
track_ids = []                                                    # Empty list to record track IDs into

for track in track_names:                                         # Heads up: with 4800 tracks to process, this takes
    song_results = sp.search(q=track, type='track', limit=1)      # awhile 
    try:
        track_ids.append(song_results['tracks']['items'][0]['id']) # Prevents program from blowing up - few tracks
        print (f"{track} song ID : {song_results['tracks']['items'][0]['id']}")   # Just to let you know it's working
    except IndexError:                                             # lack track ids
        track_ids.append(np.nan)                                   # nan if nothing

MIDDLE CHILD song ID : 2JvzF1RMd7lE3KmFlsyZD8
7 rings song ID : 6ocbgoVGwYJhOv1GgI9NsF
Sunflower - Spider-Man: Into the Spider-Verse song ID : 3KkXRkHbMCARz0aVfEt68P
Wow. song ID : 7xQAfvXzm3AkraOtGPWIZg
Going Bad (feat. Drake) song ID : 2IRZnDFmlqMuOrYOLnZZyc
Drip Too Hard (Lil Baby & Gunna) song ID : 78QR3Wp35dqAhFEc2qAGjE
Without Me song ID : 7lQ8MOhq6IN2w8EYcFNSUk
SICKO MODE song ID : 2xLMifQCjDGFmkHkpNLD9h
Look Back at It song ID : 3Ol2xnObFdKV9pmRD2t9x8
thank u, next song ID : 3e9HZxeyfWwjeyPAMmWSSQ
ZEZE (feat. Travis Scott & Offset) song ID : 0FZ4Dmg8jJJAPJnvBIzD9z
Dancing With A Stranger (with Normani) song ID : 6Qs4SXO9dwPj5GKvVOv8Ki
Mo Bamba song ID : 1xzBco0xcoJEDXktl7Jxrr
a lot song ID : 2t8yVaLvJ0RenpXUIAC52d
Leave Me Alone song ID : 6V7g6qM8wheMivMWrzbPTM
Saturday Nights song ID : 5kfNriitmkNE8mUbZ7gbq8
Wake Up in the Sky song ID : 0QIjsbm2fh1cJ45XO9eGqq
Keanu Reeves song ID : 3PYx9Wte3jwb48V0wArMOy
SAD! song ID : 3ee8Jmje8o58CHK66QrVC2
Lucid Dreams song ID : 285pBltuF7vW

Wasted (feat. Lil Uzi Vert) song ID : 1a7WZZZH7LzyvorhpOJFTe
REEL IT IN song ID : 5qHirGR7M9tdm6C17DlzSY
can't leave without it song ID : 0hPLZrnDgtKxrym1BHjDhd
Thunder song ID : 1zB4vmk8tFRmM9UULNzbLB
Uproar song ID : 3wScL5W8H40zzCKN0atfBk
Harmony Hall song ID : 39exKIvycQDgs4T6uXdyu0
Nowadays (feat. Landon Cube) song ID : 2u7mxWSeoqTXndK5e08jMp
OTW song ID : 6Hgh47WXVKtXN5zGOu0hjI
Say You Won't Let Go song ID : 5uCax9HTNlzGybIStD3vDh
Don't Come Out The House (with 21 Savage) song ID : 2Grb4G6t9VIqo6moKUloom
comethru song ID : 18uwL0vNUanqZH0ro2QcOP
the remedy for a broken heart (why am I so in love) song ID : 4UKCKdYiLN6IMA5ZESUTL7
NI BIEN NI MAL song ID : 278kSqsZIiYp8p3QjYAqa8
Te Boté - Remix song ID : 3V8UKqhEK5zBkBb6d6ub8i
I'm a Mess song ID : 04ZTP5KsCypmtCmQg5tH9R
Beautiful Crazy song ID : 2rxQMGVafnNaRaXlRMWPde
Body (feat. brando) song ID : 0BrDfbNc3HiIHPGfTT8DRc
Let You Down song ID : 52okn5MNA47tk87PeZJLEL
Call Out My Name song ID : 09mEdoA6zrmBPgTEN5qXmN
So Close song ID :

One Call song ID : 7soJgKhQTO8hLP2JPRkL5O
whoa (mind in awe) song ID : 7pdF27mSDuPWhppnHAmWHa
No Role Modelz song ID : 62vpWI1CHwFy7tMIcSStl8
Beautiful Crazy song ID : 2rxQMGVafnNaRaXlRMWPde
Congratulations song ID : 3a1lNhkSLSkpJE4MSHpDu9
XO Tour Llif3 song ID : 7GX5flRQZVHRAGd6B4TmDO
365 song ID : 2XWjPtKdi5sucFYtVav07d
BAD! song ID : 43CIWpJaJHRcI4mUSjqgc2
ocean eyes song ID : 7hDVYcQq6MxkdJGweuCtl9
KIKA song ID : 6VRwEqn7BvsN0dChfDrMAf
First Off (feat. Travis Scott) song ID : 3WRIaWsws011vHMd9uzPjG
bellyache song ID : 51NFxnQvaosfDDutk0tams
Walk Me Home song ID : 6jfEZdUJ2Yla6h7gTJgKuo
Always Remember Us This Way song ID : 2rbDhOo9Fh61Bbu23T2qCk
Sauce! song ID : 1uL78kFWxhz3umTMWJu8n5
Meant to Be (feat. Florida Georgia Line) song ID : 7iDa6hUg2VgEL1o1HjmfBn
Never Recover (Lil Baby & Gunna, Drake) song ID : 6wWaVoUOzLQJHd3bWAUpdZ
Girls Like You (feat. Cardi B) song ID : 6V1bu6o1Yo5ZXnsCJU8Ovk
goosebumps song ID : 6gBFPUFcJLzWGx4lenP6h2
monster song ID : 2UREu1Y8CO4jXkbvqAtP7g
Wit It

Startender (feat. Offset and Tyga) song ID : 0jAfdqv18goRTUxm3ilRjb
Undrunk song ID : 1Pj5oOFzLYELyyeh1IYRK0
Taste (feat. Offset) song ID : 5IaHrVsrferBYDm0bDyABy
Calling My Spirit song ID : 2MShy1GSSgbmGUxADNIao5
Beautiful Crazy song ID : 2rxQMGVafnNaRaXlRMWPde
I Like It song ID : 58q2HKrzhC3ozto2nDdN4z
Psycho (feat. Ty Dolla $ign) song ID : 3swc6WTsr7rl9DqQKQA55C
Yes Indeed song ID : 6vN77lE9LK6HP2DewaN6HZ
All Girls Are The Same song ID : 4VXIryQMWpIdGgYR4TrjT1
I Fall Apart song ID : 75ZvA4QfFiZvzhj2xkaWAh
Noticed song ID : 7zLYKWcXnYeHHWidalz7rj
Freaky song ID : 10Igtw8bSDyyFs7KIsKngZ
Who Do You Love song ID : 2fVHrSxsQbJUuj9MW9zG1e
Act Up song ID : 3A2yGHWIzmGEIolwonU69h
Ric Flair Drip (& Metro Boomin) song ID : 7sO5G9EABYOXQKNPNiE9NR
Youngblood song ID : 2iUXsYOEPhVqEBwsqP70rE
Patience song ID : 5J5CpT6bgE6KYKk23jgp9W
SLOW DANCING IN THE DARK song ID : 0rKtyWc8bvkriBthvHKY8d
Close To Me (with Diplo) (feat. Swae Lee) song ID : 5JEx7HbmvHQQswJCsoo9rA
Baila Baila Baila song ID : 7mWF

Lucid Dreams song ID : 285pBltuF7vW8TeWk8hdRR
all the good girls go to hell song ID : 6IRdLKIyS4p7XNiP8r6rsx
Outta My Head (with John Mayer) song ID : 09oZ9eXQ2fo6YDrPzJqAoP
Saturday Nights song ID : 5kfNriitmkNE8mUbZ7gbq8
Eastside (with Halsey & Khalid) song ID : 0d2iYfpKoM0QCKvcLCkBao
Here With Me song ID : 5icOoE6VgqFKohjWWNp0Ac
Thotiana song ID : 5GBuCHuPKx6UC7VsSPK0t3
Better Now song ID : 7dt6x5M1jzdTEt8oCbisTK
Shallow song ID : 2VxeLyX666F8uXCJ0dZF8B
Suge song ID : 2gwkD6igEhQbDQegRCcdoB
That's a Rack song ID : 4pCEIQ6wBVuaJdER5bALtO
Paradise song ID : 0Rx0DJI556Ix5gBny6EWmn
Let Me Down Slowly song ID : 2qxmye6gAegTMjLKEBoR3d
Mo Bamba song ID : 1xzBco0xcoJEDXktl7Jxrr
I song ID : 2kJwzbxV2ppxnQoYw4GLBZ
SAD! song ID : 3ee8Jmje8o58CHK66QrVC2
Kill This Love song ID : 6hvczQ05jc1yGlp9zhb95V
Dancing With A Stranger (with Normani) song ID : 6Qs4SXO9dwPj5GKvVOv8Ki
Racks in the Middle (feat. Roddy Ricch and Hit-Boy) song ID : 6ya8ejoKgw906Y8LWclqrp
ZEZE (feat. Travis Scott & Offset) song 

...

Don't Start Now song ID : 6WrI0LAC5M1Rw2MnX2ZvEg
death bed (coffee for your head) song ID : 7eJMfftS33KTjuF7lTsMCx
Walk Em Down (feat. Roddy Ricch) song ID : 4cSSL3YafYjM3yjgFO1vJg
Circles song ID : 21jGcNKet2qwijlDFuPiPb
Sunday Best song ID : 1Cv1YLb4q0RzL6pybtaMLo
Intentions (feat. Quavo) song ID : 4umIPjkehX1r7uhmGvXiSV
Life Is Good (feat. Drake) song ID : 5yY9lUy8nbvjM1Uyo1Uqoc
ROXANNE song ID : 696DnlkuDOXcMAnKlTgXXK
Falling song ID : 1ZMiCix7XSAbfAJlEZWMCp
Supalonely song ID : 4nK5YrxbMGZstTLbvj6Gxw
Adore You song ID : 3jjujdWJ72nww5eGnfs2E7
High Fashion (feat. Mustard) song ID : 07KXEDMj78x68D884wgVEm
After Party song ID : 527k23H0A4Q0UJN3vGs0Da
P2 song ID : 3kW4z2pIj5VY5fLjBrMRL5
Turks (feat. Travis Scott) song ID : 0lSZh5W0wDeurkGzLYY6hf
Myron song ID : 56uXDJRCuoS7abX3SkzHKQ
PTSD song ID : 3jT2LKH0RSbQ8jIUNFzXm5
Ballin' (with Roddy Ricch) song ID : 3QzAOrNlsabgbMwlZt7TAY
Sunflower - Spider-Man: Into the Spider-Verse song ID : 3KkXRkHbMCARz0aVfEt68P
Lucid Dreams song ID : 285p

Dior song ID : 79s5XnCN4TJKTVMSmOx8Ep
Enemies (feat. DaBaby) song ID : 0Xek5rqai2jcOWCYWJfVCF
Costa Rica song ID : 3tdjTdCCgKtwacsICCtPZZ
Happier song ID : 2dpaYNEQHiRxtZbfNsse99
Hurricane song ID : 6xHI9KjUjYT0FPtGO8Mxa1
Tongue Tied song ID : 0GO8y8jQk1PkHzS31d699N
Sick and Tired song ID : 0DILU9jjSTnCv1dcbUH3ib
Illuminati song ID : 2Ky6OednfOVrxDiKAhCZ7I
ORANGE SODA song ID : 5FkoSXiJPKTNyYgALRJFhD
It's You song ID : 5DqdesEfbRyOlSS3Tf6c29
Nobody But You (Duet with Gwen Stefani) song ID : 5M8goiFYynmNxhueYW6grR
i love you song ID : 6CcJMwBtXByIz4zQLzFkKc
Tusa song ID : 7k4t7uLgtOxPwTpFmtJNTY
Beautiful People (feat. Khalid) song ID : 4evmHXcjt3bTUHD1cvny97
Sucker song ID : 22vgEDb5hykfaTwLuskFGD
Save Your Tears song ID : 5QO79kh1waicV47BqGRL3g
Wait a Minute! song ID : 0y60itmpH0aPKsFiGxmtnh
GOODMORNINGTOKYO! song ID : 0Q2n5yzl2XRqYbV77tZNYp
Vete song ID : 5DxXgozhkPLgrbKFY91w0c
Lose You To Love Me song ID : 4l0Mvzj72xxOpRrp6h8nHi
Sanguine Paradise song ID : 3XiNC94b4Tq1xwv70sQJGN
Ligh

Murder On My Mind song ID : 7eBqSVxrzQZtK2mmgRG6lC
Numbers (feat. Roddy Ricch, Gunna and London On Da Track) song ID : 733c1CWmIGymoQXdp7Us88
Fuck Love (feat. Trippie Redd) song ID : 7AQim7LbvFVZJE3O8TYgf2
Lo Mein song ID : 6CUP2khYzdphXebxVTfPE3
Falling song ID : 1ZMiCix7XSAbfAJlEZWMCp
Know Your Worth song ID : 0TrPqhAMoaKUFLR7iYDokf
FIND MY WAY song ID : 1gVCEnryJhkdQcuC0Kbvor
VIBEZ song ID : 0fySG6A6qLE8IvDpayb5bM
EARFQUAKE song ID : 5hVghJ4KaYES3BFUATCYn0
JUMP (feat. YoungBoy Never Broke Again) song ID : 0oT9ElXYSxvnOOagP9efDq
No Guidance (feat. Drake) song ID : 6XHVuErjQ4XNm6nDPVCxVX
F.N song ID : 0RLLvYBopDwooCyvnTwgYu
21 song ID : 3JKgcAa7r07ocVWcV8bS0H
Tongue Tied song ID : 0GO8y8jQk1PkHzS31d699N
Señorita song ID : 0TK2YIli7K1leLovkQiNik
When It Rains It Pours song ID : 1mMLMZYXkMueg65jRRWG1l
Tonight (I Wish I Was Your Boy) song ID : 7DmTaJoM7L020qm3egqNsM
bloody valentine song ID : 6mADjHs6FXdroPzEGW6KVJ
Going Bad (feat. Drake) song ID : 2IRZnDFmlqMuOrYOLnZZyc
223's (feat. 9lo

Money In The Grave (Drake ft. Rick Ross) song ID : 5ry2OE6R2zPQFDO85XkgRb
Drip Too Hard (Lil Baby & Gunna) song ID : 78QR3Wp35dqAhFEc2qAGjE
lovely (with Khalid) song ID : 0u2P5u6lvoDfwTYjAADbn4
Truth Hurts song ID : 3HWzoMvoF3TQfYg4UPszDq
Beautiful Crazy song ID : 2rxQMGVafnNaRaXlRMWPde
Say So (feat. Nicki Minaj) song ID : 11VApNQCWLJdzxWrlmwzUa
Wow. song ID : 7xQAfvXzm3AkraOtGPWIZg
10,000 Hours (with Justin Bieber) song ID : 2wrJq5XKLnmhRXHIAf9xBa
bad guy song ID : 2Fxmhks0bxGSBdJ92vM42m
XO Tour Llif3 song ID : 7GX5flRQZVHRAGd6B4TmDO
Solitaires (feat. Travis Scott) song ID : 4kfeRwpq5KUaqTkgi4TbDF
Heartless (feat. Morgan Wallen) song ID : 3FU6urUVsgXa6RBuV2PdRk
Safaera song ID : 2DEZmgHKAvm41k4J3R2E9Y
rockstar (feat. 21 Savage) song ID : 0e7ipj03S05BNilyu5bRzt
Suge song ID : 2gwkD6igEhQbDQegRCcdoB
Sum 2 Prove song ID : 2u3EyuCqqV31woqGvO9LBz
Stunnin' (feat. Harm Franklin) song ID : 2JSmxLqt8h7ZpJJkfuLJvK
Woah song ID : 598yNsG1JFrGd4n2kMoz7F
Go Stupid song ID : 4PklAHGXT5ityno3IA8SKd


Got It On Me song ID : 25zlnOAzbVoOzIeXSvQFf1
Savage Remix (feat. Beyoncé) song ID : 5v4GgrXPMghOnBBLmveLac
Circles song ID : 21jGcNKet2qwijlDFuPiPb
After Party song ID : 527k23H0A4Q0UJN3vGs0Da
Sunflower - Spider-Man: Into the Spider-Verse song ID : 3KkXRkHbMCARz0aVfEt68P
The Bigger Picture song ID : 44gRhRi2OhEf7moAUj6MD1
Intentions (feat. Quavo) song ID : 4umIPjkehX1r7uhmGvXiSV
Rain On Me (with Ariana Grande) song ID : 24ySl2hOPGCDcxBxFIqWBu
Robbery song ID : 6Hj9jySrnFppAI0sEMCZpJ
Hit My Line song ID : 24FR8mXTaiGZPrxnkf1kcP
Stay High song ID : 21CkzgKCBCq5V4XKjiGSIj
Chicago Freestyle (feat. Giveon) song ID : 4wVOKKEHUJxHCFFNUWDn0B
GP4 song ID : 1OmmCrliFMIWzNke437KHE
Flex (feat. Juice WRLD) song ID : 2QqJTIlGKRLJC3onkavYEz
No Pressure Intro song ID : 41hwbZ1yF705MxJ7H9bwEu
All Girls Are The Same song ID : 4VXIryQMWpIdGgYR4TrjT1
RELENTLESS (feat. Lil Uzi Vert) song ID : 6nDSLDomNqfet6w0r1GHAz
Titanic song ID : 1YM6c2sxbf2Jacl9IFgn4I
Dior song ID : 79s5XnCN4TJKTVMSmOx8Ep
Bad Energy s

TIP: for this next section, add an "output_files" subfolder to export into. 

In [ ]:
# Add Track IDs to dataframe

country_tracks_df['Track ID'] = track_ids                  # Add new column with track IDs

In [ ]:
# Drop empty songs and export dataframe to csv to back it up

clean_country_tracks_df = country_tracks_df.dropna(how='any')           # Use .dropna() to remove rows with missing data

clean_country_tracks_df.to_csv("output_files/1_tracks_with_track_ids.csv", index = False)  # Back up to .csv

In [ ]:
# Continue from the backup csv file in case there is some kind of interruption to the notebook and you lose the 
# data from the API calls.

country_track_ids_df = pd.read_csv("output_files/1_tracks_with_track_ids.csv")

country_track_ids_df

In [ ]:
# Use API again to get audio features

danceability = []                                     # Set up empty lists to store data in
energy = []
valence = []
tempo = []

for track in country_track_ids_df['Track ID']:           # Heads up: this takes a long time
    
    try:
        feat_results = sp.audio_features([track])
        danceability.append(feat_results[0]['danceability'])
        energy.append(feat_results[0]['energy'])
        valence.append(feat_results[0]['valence'])
        tempo.append(feat_results[0]['tempo'])
        print (f"{track} Valence Score: {feat_results[0]['valence']}")  # Just to let you see it working                                 
    
    except TypeError:                                 # Covers you in case there is missing data
        danceability.append(np.nan)
        energy.append(np.nan)
        valence.append(np.nan)
        tempo.append(np.nan)
        

In [ ]:
# Add audio features to dataframe

country_track_ids_df['Danceability'] = danceability     # Add new columns with audio features
country_track_ids_df['Valence'] = valence
country_track_ids_df['Energy'] = energy
country_track_ids_df['Tempo'] = tempo

# Add new columns with product of % and each feature
country_track_ids_df['Danceability_Stream%'] = danceability * country_track_ids_df['Stream %']    
country_track_ids_df['Valence_Stream%'] = energy * country_track_ids_df['Stream %'] 
country_track_ids_df['Energy_Stream%'] = valence * country_track_ids_df['Stream %'] 
country_track_ids_df['Tempo_Stream%'] = tempo * country_track_ids_df['Stream %'] 

In [ ]:
# Back up dataframe again to .csv
clean_country_track_ids_df = country_track_ids_df.dropna(how='any')          # Use .dropna() to remove rows with missing data

clean_country_track_ids_df.to_csv("output_files/2_tracks_with_audio_features.csv", index=False) #Back up the dataframe to csv again

clean_country_track_ids_df

In [ ]:
# Continue from the backup csv file in case there is some kind of interruption to the notebook and you lose the 
# data from the API calls.

country_tracks_data_df = pd.read_csv("output_files/2_tracks_with_audio_features.csv")

country_tracks_data_df.head()

In [ ]:
# Use groupby to get average valence of the 200 songs in each month

country_tracks_data_groupby = country_tracks_data_df.groupby(["Year", 'Month'], as_index=False)['Valence_Stream%'].sum()

country_tracks_data_groupby

In [ ]:
# Set up some basic plt formatting configurations

plt.rc('font', size=12)
plt.rc('axes', labelsize=15)
plt.rc('axes', titlesize=20)

In [ ]:
# Plot a comparison of 2019 2020 valence scores

# Set up lists to plot

valence_2019 = country_tracks_data_groupby[country_tracks_data_groupby['Year'] == 2019]

valence_2020 = country_tracks_data_groupby[country_tracks_data_groupby['Year'] == 2020]
valence_2020.drop('Year', inplace=True, axis=1)

fig= plt.figure(figsize=(12,8))                                      # Set up figure size
fig.suptitle('SPOTIFY LISTENER VALENCE PREFERENCE BY MONTH (USA)')         # Set up main title

y_axis = valence_2019['Valence_Stream%']
x_axis = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',                  # Set up x axis
          'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

plt.plot(x_axis, valence_2019['Valence_Stream%'], label="2019 Weighted Avg Valence", marker='o', color='darkred')  # Plot 2019
plt.plot(x_axis, valence_2020['Valence_Stream%'], label="2020 Weighted Avg Valence", marker='o', color='steelblue')  # Plot 2020

plt.xlabel('Months')                                                 # Set up axis titles
plt.ylabel('Valence Score')

plt.xlim(-0.75, len(x_axis)-0.25)                                    # Set up axis limits
plt.ylim(0, 1)

plt.legend()                                                         # Include the legend

plt.show()


In [ ]:
# Compare valence scores with covide infection rate

covid_df = pd.read_csv("input_files/owid-covid-data.csv")                # read the covid data file

country_covid_df = covid_df.loc[covid_df['location'] == 'United States']   # Filter for country of your choice

country_covid_df.head()


In [ ]:
# Filter data for 2020, and add a 'month' column

country_covid__2020_df = country_covid_df[country_covid_df.date.str.contains(r'2020.*')]

country_covid__2020_df['Month'] = ''
country_covid__2020_df.head()

In [ ]:
# Extract the month from the 'date' column and add it to the new 'month' column, for sorting later

for index, row in country_covid__2020_df.iterrows():
    month, = re.findall(f"2020-(\d\d)-", row['date'])
    country_covid__2020_df.at[index, 'Month'] = int(month)  

country_covid__2020_df.head()

In [ ]:
# Create a groupby to get the sum of new cases in each month

country_covid__2020_groupby = country_covid__2020_df.groupby(['Month'], as_index=False)['new_cases'].sum()
country_covid__2020_groupby


In [ ]:
# TIP: This next section is to add missing months. In the case of New Zealand, there was no data for January
# For other countries, it might vary. Here's how I added January to the dataframe:

# No need to add january in Italy
# country_covid__2020_groupby.loc[-1] = [1, 0]             # This adds a 1 (month of January) in the first columne 
                                                         # index 0), in the last row of the dataframe. 

# country_covid__2020_groupby.index = country_covid__2020_groupby.index + 1   # shifts the index
country_covid__2020_groupby = country_covid__2020_groupby.sort_index()      # sorts by index

country_covid__2020_groupby = country_covid__2020_groupby.rename(columns={"new_cases": "New Cases"})

country_covid__2020_groupby


In [ ]:
# Merge the dataframes into one nice comparison dataframe to scatter plot

country_covid_valence_df = pd.merge(valence_2020, country_covid__2020_groupby, on="Month")

country_covid_valence_df

In [ ]:
# Add a new cases per million column

country_polulation = 5002100  #TIP: This the population of New Zealand. Adjust for your country

country_covid_valence_df['New Cases Per Million'] = country_covid_valence_df['New Cases'] / country_polulation *1000000

country_covid_valence_df

In [ ]:
# Line plot relationship between Valence scores and New Cases per Million with shared x axis and duel y axes

fig, ax1 = plt.subplots(figsize=(12,8))                   # Set up subplot figure and size
fig.suptitle('USA: SPOTIFY LISTENER VALENCE PREFERENCE BY MONTH COMPARED TO NEW COVID CASES')

months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',       # Set up shared x axis
          'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

ax1.set_xlabel('Months')                

ax1.set_ylabel('Average Valence Scores')                  # Set up first plot 
ax1.set_ylim([.2, .8])

ax1_ydata = country_covid_valence_df['Valence_Stream%']
ax1.plot(months, ax1_ydata, label="Weighted Avg Valence Scores", marker='o', color='darkred')                       
                        
ax2 = ax1.twinx()                                         # Set up second plot 
ax2.set_ylabel('New Cases Per Month')
ax2_ydata = country_covid_valence_df['New Cases Per Million']
ax2.set_ylim([0, ax2_ydata.max()+20])
ax2.plot(months, ax2_ydata, label="New Covid Cases Per Million", marker='o', color='steelblue')


lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2, loc=0)

plt.show()

In [ ]:
# This is a mini function that adds a qualitative label to the correlation r score in the scatter plot

def r_label(r):
    abs_r = abs(r)
    if abs_r >= .8 : return "Very Strong"
    elif abs_r >= .6 : return "Strong"
    elif abs_r >= .4: return "Moderate"
    elif abs_r >= .2: return "Low"
    else: return "Negligible"

In [ ]:
# Line plot relationship between Valence scores and New Cases per Million with shared x axis and duel y axes

x_axis = country_covid_valence_df['Valence_Stream%']                                 # Set up axes
y_axis = country_covid_valence_df['New Cases Per Million']

slope, intercept, rvalue, pvalue, stderr = stats.linregress(x_axis, y_axis)  # Get elements of regression equation
regress_values = x_axis * slope + intercept                                  # Calculate regression values

plt.figure(figsize=(12, 8))
plt.title('USA: SPOTIFY LISTENER VALENCE PREFERENCE VS. NEW COVID CASES') # CHANGE TITLE TO REFLECT YOUR COUNTRY

plt.xlabel(f"New Covid Cases")                                               # Set x axis label for subplot                  
plt.ylabel(f"Valence_Stream%")                                                       # Set title for subplot

r = round(stats.pearsonr(x_axis, y_axis)[0],2)                       # Calculate correlation coefficient
rlabel = r_label(r)                                                 # Call function to create a label for the r number

line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))    # Ression formula

plt.scatter(x_axis, y_axis, facecolors="darkred",  alpha=.5,        # Plot the scatter chart
label=f"r = {r}\nCorrelation is {rlabel}\n{line_eq}" ) 

plt.plot(x_axis, regress_values, color="steelblue")                 # Plot the regression line    

plt.legend()                                                        # Add the legend
    
plt.savefig("output_files/valence_vs_newcases.png")                 # Save the png file   

plt.show()